# Import Libraries

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

from art.utils import load_mnist
from art.attacks.evasion import BasicIterativeMethod
from art.defences.trainer import AdversarialTrainer
from art.estimators.classification import TensorFlowV2Classifier

# Load Data

In [2]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_mnist()

# Normal Model

In [3]:
normal_model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(10)
])

In [4]:
normal_model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [5]:
history = normal_model.fit(x_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2234 - accuracy: 0.9339
Epoch 2/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0919 - accuracy: 0.9713
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0644 - accuracy: 0.9799
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0496 - accuracy: 0.9847
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0392 - accuracy: 0.9870
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0324 - accuracy: 0.9890
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0265 - accuracy: 0.9912
Epoch 8/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0230 - accuracy: 0.9923
Epoch 9/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.0225 - accuracy: 0.9927
Epoch 10/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.015

# Adversarial Training

In [18]:
augmented_model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(256, activation='relu'),
    Dense(10)
])

In [19]:
augmented_model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [20]:
classifier = TensorFlowV2Classifier(
    model=augmented_model,
    loss_object=CategoricalCrossentropy(from_logits=True),
    nb_classes=10,
    input_shape=(28, 28),
    clip_values=(min_, max_),
    optimizer=Adam()
)

In [21]:
bim = BasicIterativeMethod(estimator=classifier, eps=0.3, eps_step=0.01, max_iter=10)

In [22]:
trainer = AdversarialTrainer(classifier=classifier, attacks=bim, ratio=0.5)

In [23]:
trainer.fit(x_train, y_train, nb_epochs=3, batch_size=32)

Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/3 [00:00<?, ?it/s]

# Attack

In [24]:
attack = BasicIterativeMethod(estimator=classifier, eps=0.3, eps_step=0.01, max_iter=10)

In [34]:
adv_test = attack.generate(x=x_test)

PGD - Batches: 0it [00:00, ?it/s]

# Results

In [35]:
normal_pred1 = np.argmax(normal_model.predict(x_test, verbose=0), axis=1)
normal_pred2 = np.argmax(normal_model.predict(adv_test, verbose=0), axis=1)

In [36]:
augmented_pred1 = np.argmax(trainer.predict(x_test, verbose=0), axis=1)
augmented_pred2 = np.argmax(trainer.predict(adv_test, verbose=0), axis=1)

In [37]:
print("Normal Model Accuracy Score on Raw Data:", np.sum(normal_pred1 == np.argmax(y_test, axis=1)) / len(normal_pred1))
print("Normal Model Accuracy Score on Adversarial Data:", np.sum(normal_pred2 == np.argmax(y_test, axis=1)) / len(normal_pred2))

Normal Model Accuracy Score on Raw Data: 0.9785
Normal Model Accuracy Score on Adversarial Data: 0.8157


In [38]:
print("Augmented Model Accuracy Score on Raw Data:", np.sum(augmented_pred1 == np.argmax(y_test, axis=1)) / len(augmented_pred1))
print("Augmented Model Accuracy Score on Adversarial Data:", np.sum(augmented_pred2 == np.argmax(y_test, axis=1)) / len(augmented_pred2))

Augmented Model Accuracy Score on Raw Data: 0.9716
Augmented Model Accuracy Score on Adversarial Data: 0.8341
